In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(float)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

###Algorithm 1 - AcceleratedAlternatingMinimization(AAM)

In [6]:
def f(x):
  return 0.5 * np.sum((X_train @ x - y_train) ** 2)

def grad_f(x):
  return X_train.T @ (X_train @ x - y_train)

def backtracking_line_search(f, grad, x, dk, alpha = 0.1, gamma = 0.8):
  beta_k = 1
  while f(x + beta_k * dk) > f(x) + alpha * beta_k * grad.T @ dk:
    beta_k *= gamma
  return beta_k

def minimize_subspace(f, grad_f, y_k, i_k, tol = 1e-6, max_iter=100):
  x_k = y_k.copy()
  for j in range(max_iter):
    grad = grad_f(x_k)
    dk = np.zeros_like(x_k)
    dk[i_k] = -grad[i_k]
    beta_k = backtracking_line_search(f, grad, x_k, dk)
    x_k += beta_k * dk

    if abs(grad[i_k]) < tol:
      break
  return x_k

In [7]:
def aam(f, grad_f, x0, v0):
  max_iter = 1000
  tol = 1e-6
  x = x0
  v = v0
  A = 0
  a = 1
  for k in range(max_iter):
    g_k = grad_f(x)
    dk = v - x
    beta_k = backtracking_line_search(f, g_k, x, dk)
    beta_k = np.clip(beta_k, 0, 1)
    y_k = x + beta_k * dk
    g_k = grad_f(y_k)
    i_k = np.argmax(g_k ** 2)
    x_k = minimize_subspace(f, grad_f, y_k, i_k)

    norm_grad_squared = np.linalg.norm(g_k) ** 2
    A_k_plus_1 = A+a
    a_k_plus_1 = np.sqrt(2 * A_k_plus_1 * (f(y_k) - f(x_k)) / norm_grad_squared)
    v = v - a_k_plus_1 * g_k
    x = x_k
    A = A_k_plus_1
    a = a_k_plus_1

    if np.linalg.norm(g_k) < tol:
      print(f"converged in {k+1} iterations")
      break

  return x

x0 = np.random.randn(X_train.shape[1])
v0 = np.zeros_like(x0)
x_opt = aam(f, grad_f, x0, v0)
y_pred = (X_test @ x_opt > 0.5).astype(float)
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


print(f"Optimal x: {x_opt}")
print(f"Test Accuracy: {accuracy}")
print(f"Test MSE: {mse}")

converged in 546 iterations
Optimal x: [ 0.03612809  0.0795792  -0.38180882 -0.04072795]
Test Accuracy: 0.9666666666666667
Test MSE: 0.03333333333333333


###Algorithm 2 - Primal-DualAAM

In [8]:
def f(x):
  return 0.5 * np.sum((X_train @ x - y_train) ** 2)

def grad_f(x):
  return X_train.T @ (X_train @ x - y_train)

def phi(lambda_):
  return 0.5 * np.sum((X_train @ lambda_ - y_train) ** 2)

def grad_phi(lambda_):
  return X_train.T @ (X_train @ lambda_ - y_train)

def backtracking_line_search(f, grad, x, d, alpha=0.1, gamma=0.8):
  beta_k = 1
  while f(x + beta_k * d) > f(x) + alpha * beta_k * grad.T @ d:
    beta_k *= gamma
  return beta_k
def minimize_subspace(f, grad_f, x, i, tol=1e-6, max_iter=1000):
  x_k = x.copy()
  for j in range(max_iter):
    g = grad_f(x_k)
    d = np.zeros_like(x_k)
    d[i] = -g[i]
    beta_k = backtracking_line_search(f, g, x_k, d)
    x_k += beta_k * d
    if abs(g[i]) < tol:
      break
  return x_k

In [9]:
def primaldual(f, grad_f, phi, grad_phi, x0, zeta0):
  max_iter = 1000
  tol = 1e-6
  x_k = x0
  zeta_k = zeta0
  A_k = 0
  a_k = 1
  eta_k = np.zeros_like(x_k)
  x_hat_k = x0

  for k in range(max_iter):
    g_k = grad_f(x_k)
    d_k = zeta_k - eta_k
    beta_k = backtracking_line_search(f, g_k, eta_k, d_k)
    beta_k = np.clip(beta_k, 0, 1)
    y_k = x_k + beta_k * d_k
    lambda_k = beta_k * zeta_k + (1 - beta_k) * eta_k
    g_lambda_k = grad_phi(lambda_k)
    i_k = np.argmax(g_lambda_k ** 2)
    x_k = minimize_subspace(f, grad_f, y_k, i_k)
    eta_k_plus_1 = minimize_subspace(phi, grad_phi, lambda_k, i_k)
    A_k_plus_1 = A_k + a_k
    norm_grad_squared = np.linalg.norm(g_lambda_k) ** 2
    a_k_plus_1 = np.sqrt(2 * A_k_plus_1 * (f(lambda_k) - f(eta_k_plus_1)) / norm_grad_squared)
    zeta_k = zeta_k - a_k_plus_1 * g_lambda_k
    x_hat_k_plus_1 = (a_k_plus_1 * lambda_k + A_k * x_hat_k) / A_k_plus_1
    x_k = x_k
    A_k = A_k_plus_1
    a_k = a_k_plus_1
    eta_k = eta_k_plus_1

    if np.linalg.norm(g_lambda_k) < tol:
      print(f"Converged in {k+1} iterations")
      break

  return x_k, eta_k

x0 = np.random.randn(X_train.shape[1])
zeta0 = np.zeros_like(x0)

x_opt, eta_opt = primaldual(f, grad_f, phi, grad_phi, x0, zeta0)
y_pred = (X_test @ x_opt > 0.5).astype(float)
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Optimal x: {x_opt}")
print(f"Optimal eta: {eta_opt}")
print(f"Test Accuracy: {accuracy}")
print(f"Test MSE: {mse}")

Optimal x: [ 0.0361173   0.07958428 -0.38177948 -0.04074548]
Optimal eta: [ 0.03612725  0.07957958 -0.38180656 -0.04072929]
Test Accuracy: 0.9666666666666667
Test MSE: 0.03333333333333333
